# 一、实验说明
参考https://github.com/Iallen520/lhy_DL_Hw/blob/master/hw1_regression.ipynb

给定训练集train.csv，测试集test.csv，要求根据前9个小时的空气检测情况预测第10个小时的PM2.5含量



要求：

现在给出的代码是这个问题的baseline，请大家：

1. 给出代码注释；

2. 想办法优化算法的结果。

3.有条件的同学，请使用这个地址参加对应的kaggle比赛 https://www.kaggle.com/c/ml2020spring-hw1/overview 
将参赛的结果页面截图粘贴到提交的代码notebook最下方。

4. 如果无法参加kaggle比赛，请修改代码，将数据集的trainning data的10%切分为测试集。进行测试。注意训练集和测试集数据不能共用。



# 二、训练集介绍

1. CSV文件，包含台湾丰原地区240天的气象观测资料，取每个月前20天的数据做训练集，每月后10天数据用于测试；
2. 每天的监测时间点为0时，1时......到23时，共24个时间节点；
3. 每天的检测指标包括CO、NO、PM2.5、PM10等气体浓度，是否降雨、刮风等气象信息，共计18项；


- train.csv部分数据展示

![jupyter](./img/traindata_eg.png)

In [1]:
import sys
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  # 引入train_test_split用于数据切分

# 三、数据预处理

浏览数据可知，数据中存在一定量的空数据NR，且多存在于RAINFALL一项。RAINFALL表示当天对应时间点是否降雨，有降雨值为1，无降雨值为NR，类似于布尔变量。因此将空数据NR全部补为0即可

In [2]:
data = pd.read_csv('/home/aistudio/data/data27964/train.csv', encoding = 'big5' ) # 读取结果的结构是DataFrame

- Pandas里主要数据结构包含DataFrame（二维表），如上打印结果，有行有列。但标准说法行（索引），列（标签）

In [3]:
# panda里利用iloc选取数据，从0开始。iloc（行，列）
# 当前选取从第三列开始的所有数据
data = data.iloc[:, 3:]
data[data=='NR'] = 0
data

,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
0,14,14,14,13,12,12,12,12,15,17,...,22,22,21,19,17,16,15,15,15,15
1,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,...,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8,1.8
2,0.51,0.41,0.39,0.37,0.35,0.3,0.37,0.47,0.78,0.74,...,0.37,0.37,0.47,0.69,0.56,0.45,0.38,0.35,0.36,0.32
3,0.2,0.15,0.13,0.12,0.11,0.06,0.1,0.13,0.26,0.23,...,0.1,0.13,0.14,0.23,0.18,0.12,0.1,0.09,0.1,0.08
4,0.9,0.6,0.5,1.7,1.8,1.5,1.9,2.2,6.6,7.9,...,2.5,2.2,2.5,2.3,2.1,1.9,1.5,1.6,1.8,1.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4315,1.8,1.8,1.8,1.8,1.8,1.7,1.7,1.8,1.8,1.8,...,1.8,1.8,2,2.1,2,1.9,1.9,1.9,2,2
4316,46,13,61,44,55,68,66,70,66,85,...,59,308,327,21,100,109,108,114,108,109
4317,36,55,72,327,74,52,59,83,106,105,...,18,311,52,54,121,97,107,118,100,105
4318,1.9,2.4,1.9,2.8,2.3,1.9,2.1,3.7,2.8,3.8,...,2.3,2.6,1.3,1,1.5,1,1.7,1.5,2,2


In [4]:
raw_data = np.array(data) # DataFrame转换成numpy数组

In [5]:
print(raw_data.shape)

(4320, 24)


In [6]:
print(raw_data)

[['14' '14' '14' ... '15' '15' '15']
 ['1.8' '1.8' '1.8' ... '1.8' '1.8' '1.8']
 ['0.51' '0.41' '0.39' ... '0.35' '0.36' '0.32']
 ...
 ['36' '55' '72' ... '118' '100' '105']
 ['1.9' '2.4' '1.9' ... '1.5' '2' '2']
 ['0.7' '0.8' '1.8' ... '1.6' '1.8' '2']]


# 四、特征提取

## （1）按月份来处理数据
- 针对每20天来说，信息维度[18, 480] (18个feature，20*24=480)
- 将原始的数据按照每个月来划分，重组成12个 [18,480]

In [7]:
month_data = {}  # key: month  value: data
for month in range(12):
    # 每月数据量
    sample = np.empty([18, 480])  # 创建一个空的【18， 480】数组
    # 每天数据量
    for day in range(20):
        # 每天24小时，对应这个18*24小时个数据
        sample[:, day * 24 : (day + 1) * 24] = raw_data[18 * (20 * month + day) : 18 * (20 * month + day + 1), :]
    month_data[month] = sample

In [8]:
# 以第一个月为例
print(month_data[0])

[[14.   14.   14.   ... 14.   13.   13.  ]
 [ 1.8   1.8   1.8  ...  1.8   1.8   1.8 ]
 [ 0.51  0.41  0.39 ...  0.34  0.41  0.43]
 ...
 [35.   79.    2.4  ... 48.   63.   53.  ]
 [ 1.4   1.8   1.   ...  1.1   1.9   1.9 ]
 [ 0.5   0.9   0.6  ...  1.2   1.2   1.3 ]]


## （2）扩充数据集，获取更好的训练效果
- 根据实验要求，需要用连续9个时间点的数据预测第10个时间点的PM2.5。 而每个月采取的是前20天连续的数据，可以扩展成480小时的连续数据；
- 具体做法，每个月的第一天的0-8时的数据作为训练数据，9时的数据作标签y；9-17的数据作一个data，18时的数据作标签y.....以此推，每个月480小时，有480-9= 471个data，故此时总数据471 * 12 个；而每个data是18*9

In [9]:
# 特征和标签矩阵初始化
x = np.empty([12 * 471, 18 * 9], dtype = float) # 特征矩阵
y = np.empty([12 * 471, 1], dtype = float) # 标签矩阵
# 构造特征和标签
for month in range(12):
    for day in range(20):
        for hour in range(24):
            # 如果是最后一天， 最后一个包结束， 则返回
            if day == 19 and hour > 14:
                continue
            # reshape将矩阵重整为新的行列数，参数-1代表自动推断,这里去掉了18*9的二维属性，
            # 转而以一维序列代替，一维序列的顺序本身可以隐含其时序信息
            # 每个小时的十八项数据
            x[month * 471 + day * 24 + hour, :] = month_data[month][:,day * 24 + hour : day * 24 + hour + 9].reshape(1, -1) 
            # pm 值
            y[month * 471 + day * 24 + hour, 0] = month_data[month][9, day * 24 + hour + 9] #value
print(x)
print(y)

[[14.  14.  14.  ...  2.   2.   0.5]
 [14.  14.  13.  ...  2.   0.5  0.3]
 [14.  13.  12.  ...  0.5  0.3  0.8]
 ...
 [17.  18.  19.  ...  1.1  1.4  1.3]
 [18.  19.  18.  ...  1.4  1.3  1.6]
 [19.  18.  17.  ...  1.3  1.6  1.8]]
[[30.]
 [41.]
 [44.]
 ...
 [17.]
 [24.]
 [29.]]


In [10]:
# 数据归一化
mean_x = np.mean(x, axis = 0) # 求均值， aix=0表示沿每列计算
std_x = np.std(x, axis = 0) # 标准差
for i in range(len(x)): 
    for j in range(len(x[0])): 
        if std_x[j] != 0:
            x[i][j] = (x[i][j] - mean_x[j]) / std_x[j] # 所有属性归一化，避免使数据的某些特征形成主导作用

In [11]:
x

array([[-1.35825331, -1.35883937, -1.359222  , ...,  0.26650729,
         0.2656797 , -1.14082131],
       [-1.35825331, -1.35883937, -1.51819928, ...,  0.26650729,
        -1.13963133, -1.32832904],
       [-1.35825331, -1.51789368, -1.67717656, ..., -1.13923451,
        -1.32700613, -0.85955971],
       ...,
       [-0.88092053, -0.72262212, -0.56433559, ..., -0.57693779,
        -0.29644471, -0.39079039],
       [-0.7218096 , -0.56356781, -0.72331287, ..., -0.29578943,
        -0.39013211, -0.1095288 ],
       [-0.56269867, -0.72262212, -0.88229015, ..., -0.38950555,
        -0.10906991,  0.07797893]])

# 损失函数
* 使用最小二乘损失函数（均方误差）来评估模型的预测效果
- num = 471*12， 乘 1/2 是为了在后续求梯度过程中保证梯度系数为1，方便计算

# 学习率更新

为了在不影响模型效果的前提下提高学习速度，可以对学习率进行实时更新：即让学习率的值在学习初期较大，之后逐渐减小。这里采用比较经典的adagrad算法来更新学习率。

In [12]:
# 切分数据集，将10%数据作为测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=42)

# 线性回归模型初始化
dim = x_train.shape[1] + 1
w = np.zeros(shape=(dim, 1))

# 添加偏置项
x_train = np.concatenate((np.ones((x_train.shape[0], 1)), x_train), axis=1).astype(float)
x_test = np.concatenate((np.ones((x_test.shape[0], 1)), x_test), axis=1).astype(float)

#初始化学习率(163个参数，163个200)和adagrad
learning_rate = np.array([[200]] * dim)
adagrad_sum = np.zeros(shape = (dim, 1 ))
 
# 训练模型
for T in range(10001):
    if T % 500 == 0:
        loss = np.sum((x_train.dot(w) - y_train) ** 2) / x_train.shape[0] / 2  # 最小二乘损失
        print(f"T={T}, Loss={loss}")

    gradient = 2 * np.transpose(x_train).dot(x_train.dot(w) - y_train)  # 损失的导数
    adagrad_sum += gradient ** 2
    w = w - learning_rate * gradient / (np.sqrt(adagrad_sum) + 0.0005)

# 保存权重
np.save('weight.npy', w)

T=0, Loss=362.8063311049941
T=500, Loss=147.61495126043522
T=1000, Loss=57.46751205761617
T=1500, Loss=35.57982054854403
T=2000, Loss=26.16381790236294
T=2500, Loss=21.596174146465565
T=3000, Loss=19.205754662001084
T=3500, Loss=17.887427897617584
T=4000, Loss=17.132549324951643
T=4500, Loss=16.687905150727318
T=5000, Loss=16.419998471305956
T=5500, Loss=16.255429511808927
T=6000, Loss=16.15255491324258
T=6500, Loss=16.087167516367597
T=7000, Loss=16.044916686599358
T=7500, Loss=16.01715230184373
T=8000, Loss=15.998583064138996
T=8500, Loss=15.985928303011043
T=9000, Loss=15.977127907047848
T=9500, Loss=15.970872289282473
T=10000, Loss=15.96631898576134


In [13]:
w.shape

(163, 1)

In [14]:
x.shape

(5652, 162)

# 使用模型预测

In [15]:
# 测试数据处理
testdata = pd.read_csv('/home/aistudio/data/data27964/test.csv', header=None, encoding='big5')
test_data = testdata.iloc[:, 2:]  # 选择测试集的相关数据
test_data[test_data == 'NR'] = 0  # 替换缺失值


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pandas/core/frame.py:3093: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


In [16]:
test_data = np.array(test_data)
test_data.shape

(4320, 9)

In [17]:

w = np.load('weight.npy')
# 构造测试特征矩阵
test_x = np.empty(shape=(240, 18 * 9), dtype=float)
for i in range(240):
    test_x[i, :] = test_data[18 * i : 18 * (i + 1), :].reshape(1, -1)

# 测试数据归一化
for i in range(test_x.shape[0]):
    for j in range(test_x.shape[1]):
        if not std_x[j] == 0:
            test_x[i][j] = (test_x[i][j] - mean_x[j]) / std_x[j]

# 添加偏置项
test_x = np.concatenate((np.ones(shape=(test_x.shape[0], 1)), test_x), axis=1).astype(float)

In [18]:
test_x.shape

(240, 163)

In [19]:
w.shape

(163, 1)

In [20]:
w = np.load('weight.npy')
ans_y = np.dot(test_x, w)

In [23]:
# 保存结果到文件
import csv
with open('submit.csv', mode='w', newline='') as submit_file:
    csv_writer = csv.writer(submit_file)
    header = ['id', 'value']
    print(header)
    csv_writer.writerow(header)
    for i in range(240):
        row = ['id_' + str(i), ans_y[i][0]]
        csv_writer.writerow(row)
        print(row)

['id', 'value']
['id_0', 7.107438020719911]
['id_1', 18.654476175271903]
['id_2', 24.134872889869182]
['id_3', 8.031828071397802]
['id_4', 26.763338709408778]
['id_5', 22.22411214749645]
['id_6', 23.526270942501338]
['id_7', 30.271792772410905]
['id_8', 16.23273345104239]
['id_9', 59.48819113158056]
['id_10', 11.679718110399447]
['id_11', 9.22279493873073]
['id_12', 62.75109352116002]
['id_13', 53.87343390571553]
['id_14', 21.689985296079534]
['id_15', 12.458915341175079]
['id_16', 32.06752063997148]
['id_17', 67.6353432039682]
['id_18', -0.4469226465384629]
['id_19', 17.06692678952303]
['id_20', 42.05176182861098]
['id_21', 71.07685563217751]
['id_22', 9.712980450966766]
['id_23', 18.068424404735502]
['id_24', 14.590673187755222]
['id_25', 38.4863705790472]
['id_26', 14.398440204614763]
['id_27', 69.80512376612637]
['id_28', 7.221055490825833]
['id_29', 55.67471792271352]
['id_30', 24.257341285234656]
['id_31', 8.4796990827337]
['id_32', 2.6302102053763075]
['id_33', 19.31248282569500